In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.python.keras import applications
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.python.keras.optimizers import RMSprop
import numpy as np
import pandas as pd

In [2]:
#parameters
classes=12
batch_size=32
train_total = 3783
validation_total = 967

In [3]:
model_vgg = VGG16(include_top=False, weights = 'imagenet',input_shape=(224,224,3))

In [4]:
model_vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
x = model_vgg.get_layer('block5_pool').output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
model = Model(inputs=model_vgg.input, outputs=x)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
for layer in model_vgg.layers:
    layer.trainable= False

In [9]:
train_datagen = ImageDataGenerator(featurewise_center=True,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
train_datagen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True)

Found 3783 images belonging to 12 classes.


In [10]:
validation_datagen = ImageDataGenerator(featurewise_center=True)
validation_datagen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 967 images belonging to 12 classes.


In [11]:
model.compile(optimizer=RMSprop(lr=0.00005),loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/10
94s - loss: 7.7239 - acc: 0.2642 - val_loss: 2.7206 - val_acc: 0.5781
Epoch 2/10
86s - loss: 5.1946 - acc: 0.4055 - val_loss: 2.1891 - val_acc: 0.6753
Epoch 3/10
87s - loss: 4.1022 - acc: 0.4696 - val_loss: 1.3264 - val_acc: 0.7404
Epoch 4/10
88s - loss: 3.5012 - acc: 0.5095 - val_loss: 1.1920 - val_acc: 0.7580
Epoch 5/10
89s - loss: 2.9644 - acc: 0.5443 - val_loss: 1.0265 - val_acc: 0.7766
Epoch 6/10
88s - loss: 2.4826 - acc: 0.5862 - val_loss: 1.1561 - val_acc: 0.7446
Epoch 7/10
89s - loss: 2.4174 - acc: 0.6096 - val_loss: 1.0036 - val_acc: 0.7611
Epoch 8/10
89s - loss: 2.1191 - acc: 0.6188 - val_loss: 0.8190 - val_acc: 0.7797
Epoch 9/10
90s - loss: 1.9647 - acc: 0.6206 - val_loss: 0.7562 - val_acc: 0.7839
Epoch 10/10
88s - loss: 1.8184 - acc: 0.6423 - val_loss: 0.7152 - val_acc: 0.8087


In [13]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/10
93s - loss: 1.7359 - acc: 0.6653 - val_loss: 0.6622 - val_acc: 0.8304
Epoch 2/10
85s - loss: 1.6349 - acc: 0.6701 - val_loss: 0.6657 - val_acc: 0.8087
Epoch 3/10
85s - loss: 1.5661 - acc: 0.6795 - val_loss: 0.7148 - val_acc: 0.8190
Epoch 4/10
86s - loss: 1.4964 - acc: 0.6881 - val_loss: 0.6354 - val_acc: 0.8128
Epoch 5/10
85s - loss: 1.3734 - acc: 0.7031 - val_loss: 0.6583 - val_acc: 0.7942
Epoch 6/10
85s - loss: 1.3812 - acc: 0.7016 - val_loss: 0.6001 - val_acc: 0.8252
Epoch 7/10
86s - loss: 1.3127 - acc: 0.7079 - val_loss: 0.6383 - val_acc: 0.8118
Epoch 8/10
87s - loss: 1.2700 - acc: 0.7282 - val_loss: 0.6143 - val_acc: 0.8294
Epoch 9/10
85s - loss: 1.1912 - acc: 0.7330 - val_loss: 0.4875 - val_acc: 0.8542
Epoch 10/10
85s - loss: 1.1479 - acc: 0.7404 - val_loss: 0.5932 - val_acc: 0.8459


In [14]:
model.compile(optimizer=RMSprop(lr=0.000005),loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/20
94s - loss: 1.0809 - acc: 0.7494 - val_loss: 0.5440 - val_acc: 0.8532
Epoch 2/20
85s - loss: 1.1658 - acc: 0.7396 - val_loss: 0.4973 - val_acc: 0.8542
Epoch 3/20
84s - loss: 1.0011 - acc: 0.7554 - val_loss: 0.4874 - val_acc: 0.8604
Epoch 4/20
86s - loss: 1.0556 - acc: 0.7543 - val_loss: 0.5246 - val_acc: 0.8490
Epoch 5/20
85s - loss: 0.9822 - acc: 0.7655 - val_loss: 0.4780 - val_acc: 0.8594
Epoch 6/20
84s - loss: 0.9808 - acc: 0.7605 - val_loss: 0.4769 - val_acc: 0.8542
Epoch 7/20
84s - loss: 0.9533 - acc: 0.7732 - val_loss: 0.4945 - val_acc: 0.8532
Epoch 8/20
84s - loss: 0.9222 - acc: 0.7677 - val_loss: 0.4995 - val_acc: 0.8532
Epoch 9/20
85s - loss: 0.9428 - acc: 0.7699 - val_loss: 0.4755 - val_acc: 0.8573
Epoch 10/20
87s - loss: 0.9527 - acc: 0.7701 - val_loss: 0.4759 - val_acc: 0.8469
Epoch 11/20
86s - loss: 0.8537 - acc: 0.7815 - val_loss: 0.4517 - val_acc: 0.8573
Epoch 12/20
84s - loss: 0.8578 - acc: 0.7799 - val_loss: 0.5245 - val_acc: 0.8428
Epoch 13/20
89s - loss: 0

In [16]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/20
101s - loss: 0.8114 - acc: 0.7836 - val_loss: 0.4425 - val_acc: 0.8532
Epoch 2/20
92s - loss: 0.7891 - acc: 0.7842 - val_loss: 0.4612 - val_acc: 0.8583
Epoch 3/20
93s - loss: 0.8009 - acc: 0.7871 - val_loss: 0.4301 - val_acc: 0.8697
Epoch 4/20
86s - loss: 0.7662 - acc: 0.7917 - val_loss: 0.4597 - val_acc: 0.8449
Epoch 5/20
88s - loss: 0.7857 - acc: 0.7865 - val_loss: 0.4547 - val_acc: 0.8656
Epoch 6/20
85s - loss: 0.7956 - acc: 0.7853 - val_loss: 0.4288 - val_acc: 0.8532
Epoch 7/20
86s - loss: 0.7216 - acc: 0.8100 - val_loss: 0.4555 - val_acc: 0.8542
Epoch 8/20
85s - loss: 0.7152 - acc: 0.7946 - val_loss: 0.4510 - val_acc: 0.8614
Epoch 9/20
87s - loss: 0.7562 - acc: 0.7961 - val_loss: 0.4389 - val_acc: 0.8583
Epoch 10/20
89s - loss: 0.7448 - acc: 0.8018 - val_loss: 0.4565 - val_acc: 0.8656
Epoch 11/20
88s - loss: 0.7471 - acc: 0.7985 - val_loss: 0.4741 - val_acc: 0.8490
Epoch 12/20
85s - loss: 0.7528 - acc: 0.8004 - val_loss: 0.4498 - val_acc: 0.8594
Epoch 13/20
86s - loss: 

KeyboardInterrupt: 

In [12]:
model.compile(optimizer=RMSprop(lr=0.000001),loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/10
95s - loss: 0.7238 - acc: 0.8116 - val_loss: 0.4426 - val_acc: 0.8656
Epoch 2/10
88s - loss: 0.7671 - acc: 0.7951 - val_loss: 0.4362 - val_acc: 0.8625
Epoch 3/10
87s - loss: 0.7005 - acc: 0.8059 - val_loss: 0.4484 - val_acc: 0.8635
Epoch 4/10
86s - loss: 0.7976 - acc: 0.7926 - val_loss: 0.4583 - val_acc: 0.8625
Epoch 5/10


KeyboardInterrupt: 

In [19]:
model.save_weights('vgg16_fixed_bottom_temp.hdf5')

In [11]:
model.load_weights('vgg16_fixed_bottom_temp.hdf5')

In [12]:
for layer in model_vgg.layers[15:19]:
    layer.trainable= True

In [14]:
model.compile(optimizer=RMSprop(lr=1e-9),loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/10
94s - loss: 0.7239 - acc: 0.7980 - val_loss: 0.4282 - val_acc: 0.8707
Epoch 2/10
85s - loss: 0.7333 - acc: 0.8041 - val_loss: 0.4154 - val_acc: 0.8604
Epoch 3/10
86s - loss: 0.7438 - acc: 0.7993 - val_loss: 0.4583 - val_acc: 0.8656
Epoch 4/10
88s - loss: 0.6932 - acc: 0.8026 - val_loss: 0.4277 - val_acc: 0.8583
Epoch 5/10
89s - loss: 0.7313 - acc: 0.8064 - val_loss: 0.4334 - val_acc: 0.8656
Epoch 6/10
86s - loss: 0.6751 - acc: 0.8051 - val_loss: 0.4493 - val_acc: 0.8583
Epoch 7/10
87s - loss: 0.6771 - acc: 0.8125 - val_loss: 0.4371 - val_acc: 0.8625
Epoch 8/10
84s - loss: 0.7077 - acc: 0.8110 - val_loss: 0.4708 - val_acc: 0.8604
Epoch 9/10
86s - loss: 0.7064 - acc: 0.7974 - val_loss: 0.4108 - val_acc: 0.8635
Epoch 10/10
85s - loss: 0.7479 - acc: 0.7950 - val_loss: 0.4307 - val_acc: 0.8625


In [15]:
model.compile(optimizer=RMSprop(lr=1e-),loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/10
93s - loss: 0.6707 - acc: 0.8151 - val_loss: 0.3989 - val_acc: 0.8749
Epoch 2/10
88s - loss: 0.6685 - acc: 0.8142 - val_loss: 0.3996 - val_acc: 0.8707
Epoch 3/10
91s - loss: 0.6261 - acc: 0.8118 - val_loss: 0.4404 - val_acc: 0.8563
Epoch 4/10
88s - loss: 0.6399 - acc: 0.8180 - val_loss: 0.4070 - val_acc: 0.8635
Epoch 5/10
89s - loss: 0.6593 - acc: 0.8183 - val_loss: 0.3943 - val_acc: 0.8759
Epoch 6/10
92s - loss: 0.6618 - acc: 0.8110 - val_loss: 0.3884 - val_acc: 0.8676
Epoch 7/10
89s - loss: 0.5995 - acc: 0.8236 - val_loss: 0.4109 - val_acc: 0.8707
Epoch 8/10
88s - loss: 0.5989 - acc: 0.8218 - val_loss: 0.3544 - val_acc: 0.8831
Epoch 9/10
88s - loss: 0.5234 - acc: 0.8356 - val_loss: 0.3673 - val_acc: 0.8800
Epoch 10/10
92s - loss: 0.5639 - acc: 0.8270 - val_loss: 0.4039 - val_acc: 0.8697


In [17]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/10
95s - loss: 0.4951 - acc: 0.8427 - val_loss: 0.3304 - val_acc: 0.8904
Epoch 2/10
87s - loss: 0.5598 - acc: 0.8301 - val_loss: 0.3884 - val_acc: 0.8769
Epoch 3/10
91s - loss: 0.5363 - acc: 0.8419 - val_loss: 0.3499 - val_acc: 0.8842
Epoch 4/10
87s - loss: 0.5128 - acc: 0.8446 - val_loss: 0.3766 - val_acc: 0.8687
Epoch 5/10
87s - loss: 0.5005 - acc: 0.8461 - val_loss: 0.3633 - val_acc: 0.8790
Epoch 6/10
87s - loss: 0.5074 - acc: 0.8440 - val_loss: 0.3338 - val_acc: 0.8904
Epoch 7/10
88s - loss: 0.5196 - acc: 0.8334 - val_loss: 0.3532 - val_acc: 0.8852
Epoch 8/10
86s - loss: 0.5102 - acc: 0.8382 - val_loss: 0.3467 - val_acc: 0.8935
Epoch 9/10
90s - loss: 0.4782 - acc: 0.8499 - val_loss: 0.3380 - val_acc: 0.8893
Epoch 10/10
86s - loss: 0.5281 - acc: 0.8412 - val_loss: 0.3517 - val_acc: 0.8873


In [13]:
model.compile(optimizer=RMSprop(lr=1e-5),loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/10
95s - loss: 0.5078 - acc: 0.8427 - val_loss: 0.3173 - val_acc: 0.8987
Epoch 2/10
87s - loss: 0.4728 - acc: 0.8469 - val_loss: 0.3206 - val_acc: 0.8904
Epoch 3/10
89s - loss: 0.4362 - acc: 0.8603 - val_loss: 0.3568 - val_acc: 0.8862
Epoch 4/10
91s - loss: 0.4272 - acc: 0.8636 - val_loss: 0.3750 - val_acc: 0.8759
Epoch 5/10
90s - loss: 0.4187 - acc: 0.8628 - val_loss: 0.3255 - val_acc: 0.8873
Epoch 6/10
91s - loss: 0.4133 - acc: 0.8684 - val_loss: 0.3439 - val_acc: 0.8728
Epoch 7/10
90s - loss: 0.3692 - acc: 0.8718 - val_loss: 0.3183 - val_acc: 0.8873
Epoch 8/10
89s - loss: 0.3609 - acc: 0.8822 - val_loss: 0.2874 - val_acc: 0.8987
Epoch 9/10
88s - loss: 0.3293 - acc: 0.8901 - val_loss: 0.3442 - val_acc: 0.8914
Epoch 10/10
90s - loss: 0.3474 - acc: 0.8823 - val_loss: 0.2819 - val_acc: 0.9131


In [15]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/20
96s - loss: 0.3120 - acc: 0.8965 - val_loss: 0.3099 - val_acc: 0.9069
Epoch 2/20
88s - loss: 0.3358 - acc: 0.8860 - val_loss: 0.2457 - val_acc: 0.9235
Epoch 3/20
89s - loss: 0.3180 - acc: 0.8965 - val_loss: 0.2860 - val_acc: 0.8997
Epoch 4/20
89s - loss: 0.2961 - acc: 0.8993 - val_loss: 0.2723 - val_acc: 0.9255
Epoch 5/20
102s - loss: 0.2801 - acc: 0.9047 - val_loss: 0.2487 - val_acc: 0.9245
Epoch 6/20
128s - loss: 0.2853 - acc: 0.9034 - val_loss: 0.2856 - val_acc: 0.9162
Epoch 7/20
130s - loss: 0.2759 - acc: 0.9031 - val_loss: 0.2975 - val_acc: 0.9080
Epoch 8/20
123s - loss: 0.2736 - acc: 0.9160 - val_loss: 0.2254 - val_acc: 0.9297
Epoch 9/20
129s - loss: 0.2411 - acc: 0.9183 - val_loss: 0.2812 - val_acc: 0.9162
Epoch 10/20
123s - loss: 0.2651 - acc: 0.9125 - val_loss: 0.2475 - val_acc: 0.9266
Epoch 11/20


KeyboardInterrupt: 

In [13]:
model.compile(optimizer=RMSprop(lr=1e-7),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/20
92s - loss: 0.4233 - acc: 0.8678 - val_loss: 0.3039 - val_acc: 0.9028
Epoch 2/20
85s - loss: 0.3927 - acc: 0.8724 - val_loss: 0.2888 - val_acc: 0.8997
Epoch 3/20
84s - loss: 0.4302 - acc: 0.8671 - val_loss: 0.2917 - val_acc: 0.9018
Epoch 4/20


In [ ]:
model.save_weights('vgg16_fixed_bottom_temp.hdf5')

In [19]:
validation_datagen = ImageDataGenerator(featurewise_center=True)
validation_datagen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

test_generator = validation_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode=None)

Found 794 images belonging to 1 classes.


In [26]:
batch_size

32

In [27]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))


In [28]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
class_ids

{0: 'Black-grass',
 1: 'Charlock',
 2: 'Cleavers',
 3: 'Common Chickweed',
 4: 'Common wheat',
 5: 'Fat Hen',
 6: 'Loose Silky-bent',
 7: 'Maize',
 8: 'Scentless Mayweed',
 9: 'Shepherds Purse',
 10: 'Small-flowered Cranesbill',
 11: 'Sugar beet'}

In [29]:
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [30]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]

In [31]:
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_vgg_16_fixed_bottom.csv', encoding="utf8", index=False)

In [32]:
from IPython.display import FileLink
FileLink('submission_vgg_16_fixed_bottom.csv')

/home/janardhan/plant_seeding/submission_vgg_16_fixed_bottom.csv